# Preprocess bloom peak datasets

In [297]:
from meteostat import Stations, Point, Daily, Monthly
import math
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from datetime import datetime
from random import sample
import matplotlib.pyplot as plt

In [315]:
# bloom peak dates between Japan/Kyoto and Kyoto do not match. Same for Switzerland/Liestal and liestal. 
# Thus, we need to make a dicision waht to choose as ground truth. 
# Since weather data, including daily precipitation, for Japan are available from 1951, I chose data from japan.csv
filenames = ['data/vancouver.csv', 'data/japan.csv', 'data/kyoto.csv', 'data/meteoswiss.csv', 'data/liestal.csv',
             'data/south_korea.csv', 'data/washingtondc.csv']
df_bloom = pd.concat([pd.read_csv(f) for f in filenames])
df_bloom = df_bloom.reset_index(drop=True)  # make sure indexes pair with number of rows
print('Number of observations: ', df_bloom.shape[0])
print(df_bloom.dtypes)
df_bloom.head(4)

Number of observations:  15146
location       object
lat           float64
long           object
alt           float64
year            int64
bloom_date     object
bloom_doy       int64
dtype: object


,location,lat,long,alt,year,bloom_date,bloom_doy
0,vancouver,49.2237,–123.1636,24.00,2022,2022-03-27,86
1,Japan/Wakkanai,45.4150,141.678889,2.85,1953,1953-05-30,150
2,Japan/Wakkanai,45.4150,141.678889,2.85,1954,1954-05-27,147
3,Japan/Wakkanai,45.4150,141.678889,2.85,1955,1955-05-23,143


In [63]:
#choose years randomly to use as validation set
common_years = set(df_bloom.year[df_bloom.location=='liestal'])& \
set(df_bloom.year[df_bloom.location=='washingtondc'])& \
set(df_bloom.year[df_bloom.location=='Japan/Kyoto']) #weather data, including daily precipitation, are available from 1951
print('Length of set: ', len(common_years))
print('Validation set length: ', len(common_years)*0.05)
print(sample(common_years,3)

Length of set:  69
Validation set length:  3.45
[2020, 1970, 1979]


since Python 3.9 and will be removed in a subsequent version.


In [ ]:
val_year = [1970, 1979, 2020]

## Extend dataset with weather info

In [299]:
# 1. Extend dataset with weather info staring from January 1 till bloom peak date for each year
# 2. The earliest bllom day is January 8. Keep a week before to calculate heat accumulation
# 3. Assign classes 0 to 7 for each day before the bllom day. Zero class for more than 7 days before bloom peak
# Weather availability: Washington DC since 1942; Switzerland since 1901; Japan since 1951.
df_bloom.bloom_doy.describe()

count    15146.000000
mean       106.858312
std         19.220305
min          8.000000
25%         96.000000
50%        105.000000
75%        118.000000
max        175.000000
Name: bloom_doy, dtype: float64

In [380]:
# skip 'Japan/Kyoto'; 'Switzerland/Liestal' to avoid dublications
locations = set(df_bloom.location) - set(["Japan/Kyoto", "Switzerland/Liestal"])
print("Unique locations: ", len(locations))
df_bloom_unq = df_bloom[df_bloom.location.isin(locations)]

# clean data: convert string to datatime; correct minus sign
df_bloom_unq.bloom_date = df_bloom_unq.bloom_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())
df_bloom_unq.long =  df_bloom_unq.long.apply(lambda x: x if type(x)==float else float(x.replace('\U00002013', '-')))
print(df_bloom_unq.shape)
print(df_bloom_unq.dtypes)

Unique locations:  327
(15039, 7)
location       object
lat           float64
long          float64
alt           float64
year            int64
bloom_date     object
bloom_doy       int64
dtype: object


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Using daily data for classification

In [292]:
def get_weather(row, cols):
    # get dataframe with weather info
    start = datetime(row.year, 1, 1) # from January 1st
    end = datetime(row.year, row.bloom_date.month, row.bloom_date.day) # Untill bloom peak
    long = row.long if type(row.long)==float else float(row.long.replace('\U00002013', '-'))
    location = Point(row.lat, long, row.alt) 
    try:
        df_weather = Daily(location, start, end)
    except Exception as e:
        print("No data for ", row.location, ' at ', row.year)
        return pd.DataFrame()

    df_weather = df_weather.fetch()
    #if there are no precipitation for this year, skip it
    if df_weather.shape[0]==0:
        print("No data for ", row.location, ' at ', row.year)
        return pd.DataFrame()
    if df_weather.prcp.first_valid_index()==None:
        print("No precipitation data for ", row.location, " at ", row.year)
        return pd.DataFrame()
    df_weather["day"]=df_weather.index
    
    #sometimes available only average temperature; sometimes  only minimum and maximum temperature
    #replace NA's for average temperature, using minimum and maximum temperature
    df_weather.tavg.fillna((df_weather.tmin+ df_weather.tmax)/2, inplace=True)
    #replace NA's for minimum and maximum temperature using average temperature
    df_weather["tmin"].fillna(df_weather.tavg, inplace=True)
    df_weather["tmax"].fillna(df_weather.tavg, inplace=True)
    
    # add new features
    df_weather.loc[:,'3day_mean'] = df_weather.loc[:,'tavg'].rolling(3).mean(skipna=True)
    df_weather.loc[:,'7day_mean'] = df_weather.loc[:,'tavg'].rolling(7).mean(skipna=True)
    df_weather['heat_sum'] = df_weather['tavg'].cumsum(skipna=True)

    # merge with location
    df_loc = pd.DataFrame(columns=cols)
    df_loc.loc[0] = row
    df_loc_rep = pd.concat([pd.DataFrame(df_loc.values, columns=cols)]*df_weather.shape[0], ignore_index=True)
    df_loc_rep.index = df_weather.index
    df = pd.concat([df_loc_rep, df_weather], axis=1)
    # add class labels
    df['class'] = 0
    try:
        df.loc[df['bloom_date'] == df['day'], 'class'] = 7
        df.loc[(df['bloom_date']-timedelta(days=1))== df['day'], 'class'] = 6
        df.loc[(df['bloom_date']-timedelta(days=2))== df['day'], 'class'] = 5
        df.loc[(df['bloom_date']-timedelta(days=3))== df['day'], 'class'] = 4
        df.loc[(df['bloom_date']-timedelta(days=4))== df['day'], 'class'] = 3
        df.loc[(df['bloom_date']-timedelta(days=5))== df['day'], 'class'] = 2
        df.loc[(df['bloom_date']-timedelta(days=6))== df['day'], 'class'] = 1
    except Exception as e:
        print("No data for ", row.location, ' at ', row.year)
        return pd.DataFrame()  
    return df

In [293]:
df_ext = pd.DataFrame()
for index, row in df_bloom_unq.iterrows():
    #print(row.location, row.year)
    df_cur = get_weather(row, df_bloom_unq.columns)
    df_ext = df_ext.append(df_cur, ignore_index=True)
    #if index>0:
    #break

No data for  Japan/Wakkanai  at  2005
No data for  Japan/Rumoi  at  2005
No data for  Japan/Asahikawa  at  2005
No data for  Japan/Abashiri  at  2005
No data for  Japan/Iwamizawa  at  1990
No data for  Japan/Iwamizawa  at  2005
No data for  Japan/Obihiro  at  2005
No data for  Japan/Kushiro  at  2005
No data for  Japan/Kushiro  at  2005
No data for  Japan/Nemuro  at  2005
No data for  Japan/Muroran  at  2005
No data for  Japan/Muroran  at  2005
No data for  Japan/Urakawa  at  2005
No data for  Japan/Esashi  at  2005
No data for  Japan/Hakodate  at  2005
No data for  Japan/Kutchan  at  1990
No data for  Japan/Kutchan  at  2005
No data for  Japan/Mombetsu  at  1990
No data for  Japan/Mombetsu  at  2005
No data for  Japan/Hiroo  at  1990
No data for  Japan/Hiroo  at  1991
No data for  Japan/Shinjo  at  1990
No data for  Japan/Shinjo  at  1991
No data for  Japan/Aomori  at  2005
No precipitation data for  Japan/Hachinohe  at  2005
No data for  Japan/Akita  at  2005
No data for  Japan/Akita

No data for  Japan/Hachijojima  at  2005
No data for  Japan/Saigo  at  2005
No data for  Japan/Tottori  at  2005
No data for  Japan/Tottori  at  2005
No data for  Japan/Toyooka  at  1990
No data for  Japan/Toyooka  at  1991
No data for  Japan/Toyooka  at  1992
No data for  Japan/Toyooka  at  1993
No data for  Japan/Toyooka  at  1994
No data for  Japan/Toyooka  at  1995
No data for  Japan/Toyooka  at  1996
No precipitation data for  Japan/Toyooka  at  1997
No precipitation data for  Japan/Toyooka  at  1998
No precipitation data for  Japan/Toyooka  at  1999
No precipitation data for  Japan/Toyooka  at  2000
No precipitation data for  Japan/Toyooka  at  2001
No precipitation data for  Japan/Toyooka  at  2002
No precipitation data for  Japan/Toyooka  at  2003
No precipitation data for  Japan/Toyooka  at  2004
No precipitation data for  Japan/Toyooka  at  2005
No precipitation data for  Japan/Toyooka  at  2006
No precipitation data for  Japan/Toyooka  at  2007
No data for  Japan/Maizuru  at

No precipitation data for  Japan/Minamidaitojima  at  2011
No precipitation data for  Japan/Minamidaitojima  at  2012
No precipitation data for  Japan/Minamidaitojima  at  2013
No precipitation data for  Japan/Minamidaitojima  at  2014
No precipitation data for  Japan/Minamidaitojima  at  2015
No precipitation data for  Japan/Minamidaitojima  at  2016
No precipitation data for  Japan/Minamidaitojima  at  2017
No precipitation data for  Japan/Minamidaitojima  at  2018
No precipitation data for  Japan/Minamidaitojima  at  2019
No data for  kyoto  at  812
No data for  kyoto  at  815
No data for  kyoto  at  831
No data for  kyoto  at  851
No data for  kyoto  at  853
No data for  kyoto  at  864
No data for  kyoto  at  866
No data for  kyoto  at  869
No data for  kyoto  at  889
No data for  kyoto  at  891
No data for  kyoto  at  892
No data for  kyoto  at  894
No data for  kyoto  at  895
No data for  kyoto  at  896
No data for  kyoto  at  902
No data for  kyoto  at  908
No data for  kyoto  a

No data for  kyoto  at  1422
No data for  kyoto  at  1423
No data for  kyoto  at  1424
No data for  kyoto  at  1425
No data for  kyoto  at  1426
No data for  kyoto  at  1427
No data for  kyoto  at  1429
No data for  kyoto  at  1430
No data for  kyoto  at  1431
No data for  kyoto  at  1432
No data for  kyoto  at  1433
No data for  kyoto  at  1434
No data for  kyoto  at  1435
No data for  kyoto  at  1436
No data for  kyoto  at  1437
No data for  kyoto  at  1438
No data for  kyoto  at  1439
No data for  kyoto  at  1440
No data for  kyoto  at  1441
No data for  kyoto  at  1443
No data for  kyoto  at  1444
No data for  kyoto  at  1446
No data for  kyoto  at  1447
No data for  kyoto  at  1448
No data for  kyoto  at  1449
No data for  kyoto  at  1450
No data for  kyoto  at  1451
No data for  kyoto  at  1452
No data for  kyoto  at  1453
No data for  kyoto  at  1454
No data for  kyoto  at  1455
No data for  kyoto  at  1456
No data for  kyoto  at  1457
No data for  kyoto  at  1458
No data for  k

No data for  kyoto  at  1736
No data for  kyoto  at  1737
No data for  kyoto  at  1738
No data for  kyoto  at  1739
No data for  kyoto  at  1740
No data for  kyoto  at  1741
No data for  kyoto  at  1742
No data for  kyoto  at  1743
No data for  kyoto  at  1744
No data for  kyoto  at  1745
No data for  kyoto  at  1746
No data for  kyoto  at  1747
No data for  kyoto  at  1748
No data for  kyoto  at  1749
No data for  kyoto  at  1750
No data for  kyoto  at  1751
No data for  kyoto  at  1752
No data for  kyoto  at  1753
No data for  kyoto  at  1754
No data for  kyoto  at  1755
No data for  kyoto  at  1756
No data for  kyoto  at  1757
No data for  kyoto  at  1758
No data for  kyoto  at  1759
No data for  kyoto  at  1760
No data for  kyoto  at  1761
No data for  kyoto  at  1762
No data for  kyoto  at  1763
No data for  kyoto  at  1764
No data for  kyoto  at  1765
No data for  kyoto  at  1766
No data for  kyoto  at  1767
No data for  kyoto  at  1768
No data for  kyoto  at  1769
No data for  k

...

No precipitation data for  Switzerland/Altdorf  at  2020
No precipitation data for  Switzerland/Andeer  at  2020
No precipitation data for  Switzerland/Ballens  at  2020
No precipitation data for  Switzerland/Bellelay  at  2020
No precipitation data for  Switzerland/Blonay  at  2020
No precipitation data for  Switzerland/Biel  at  2020
No precipitation data for  Switzerland/Bondo GR  at  2020
No precipitation data for  Switzerland/La Brévine  at  2020
No precipitation data for  Switzerland/Chardonne  at  2020
No precipitation data for  Switzerland/Chaumont  at  2020
No precipitation data for  Switzerland/Disentis  at  2020
No precipitation data for  Switzerland/Escholzmatt  at  2020
No precipitation data for  Switzerland/Elm  at  2020
No precipitation data for  Switzerland/Faido  at  2020
No precipitation data for  Switzerland/Fiesch  at  2020
No precipitation data for  Switzerland/Gadmen  at  2020
No precipitation data for  Switzerland/Gstaad  at  2020
No precipitation data for  Switz

No data for  South Korea/Daegwallyeong  at  1991
No data for  South Korea/Daegwallyeong  at  1992
No data for  South Korea/Daegwallyeong  at  1993
No data for  South Korea/Daegwallyeong  at  1994
No precipitation data for  South Korea/Chuncheon  at  1981
No precipitation data for  South Korea/Gangneung  at  1980
No precipitation data for  South Korea/Gangneung  at  1981
No data for  South Korea/Wonju  at  1980
No data for  South Korea/Wonju  at  1981
No data for  South Korea/Wonju  at  1982
No precipitation data for  South Korea/Wonju  at  1983
No data for  South Korea/Wonju  at  1984
No data for  South Korea/Wonju  at  1985
No data for  South Korea/Wonju  at  1986
No data for  South Korea/Wonju  at  1987
No data for  South Korea/Wonju  at  1988
No data for  South Korea/Wonju  at  1989
No data for  South Korea/Wonju  at  1990
No data for  South Korea/Wonju  at  1991
No data for  South Korea/Wonju  at  1992
No data for  South Korea/Wonju  at  1993
No precipitation data for  South Korea/

No precipitation data for  South Korea/Daegu  at  2019
No precipitation data for  South Korea/Daegu  at  2020
No data for  South Korea/Jeonju  at  1980
No data for  South Korea/Jeonju  at  1981
No data for  South Korea/Jeonju  at  1982
No data for  South Korea/Jeonju  at  1984
No data for  South Korea/Jeonju  at  1985
No data for  South Korea/Jeonju  at  1986
No data for  South Korea/Jeonju  at  1987
No data for  South Korea/Jeonju  at  1988
No data for  South Korea/Jeonju  at  1989
No data for  South Korea/Jeonju  at  1990
No data for  South Korea/Jeonju  at  1991
No data for  South Korea/Jeonju  at  1992
No data for  South Korea/Jeonju  at  1993
No data for  South Korea/Jeonju  at  1994
No data for  South Korea/Jeonju  at  2015
No data for  South Korea/Jeonju  at  2016
No data for  South Korea/Jeonju  at  2017
No data for  South Korea/Jeonju  at  2018
No precipitation data for  South Korea/Jeonju  at  2019
No precipitation data for  South Korea/Jeonju  at  2020
No data for  South Kor

No data for  South Korea/Jecheon  at  1993
No precipitation data for  South Korea/Jecheon  at  1994
No data for  South Korea/Boeun  at  1980
No precipitation data for  South Korea/Boeun  at  1981
No data for  South Korea/Boeun  at  1982
No precipitation data for  South Korea/Boeun  at  1983
No precipitation data for  South Korea/Boeun  at  1984
No data for  South Korea/Boeun  at  1985
No data for  South Korea/Boeun  at  1986
No data for  South Korea/Boeun  at  1987
No precipitation data for  South Korea/Boeun  at  1988
No data for  South Korea/Boeun  at  1989
No data for  South Korea/Boeun  at  1990
No data for  South Korea/Boeun  at  1991
No data for  South Korea/Boeun  at  1992
No data for  South Korea/Boeun  at  1993
No precipitation data for  South Korea/Boeun  at  1994
No precipitation data for  South Korea/Cheonan  at  1980
No precipitation data for  South Korea/Cheonan  at  1981
No precipitation data for  South Korea/Cheonan  at  1982
No precipitation data for  South Korea/Cheon

No data for  South Korea/Yeongdeok  at  1988
No data for  South Korea/Yeongdeok  at  1989
No data for  South Korea/Yeongdeok  at  1990
No data for  South Korea/Yeongdeok  at  1991
No data for  South Korea/Yeongdeok  at  1992
No data for  South Korea/Yeongdeok  at  1993
No data for  South Korea/Yeongdeok  at  1994
No data for  South Korea/Yeongdeok  at  1995
No data for  South Korea/Uiseong  at  1980
No data for  South Korea/Uiseong  at  1981
No data for  South Korea/Uiseong  at  1982
No data for  South Korea/Uiseong  at  1983
No data for  South Korea/Uiseong  at  1984
No data for  South Korea/Uiseong  at  1985
No data for  South Korea/Uiseong  at  1986
No data for  South Korea/Uiseong  at  1987
No data for  South Korea/Uiseong  at  1988
No data for  South Korea/Uiseong  at  1989
No data for  South Korea/Uiseong  at  1990
No data for  South Korea/Uiseong  at  1991
No data for  South Korea/Uiseong  at  1992
No data for  South Korea/Uiseong  at  1993
No data for  South Korea/Uiseong  at  

In [294]:
df_ext.tail()

,location,lat,long,alt,year,bloom_date,bloom_doy,tavg,tmin,tmax,...,wdir,wspd,wpgt,pres,tsun,day,3day_mean,7day_mean,heat_sum,class
991236,washingtondc,38.88535,-77.038628,0.0,2022,2022-03-21,80,5.1,-4.3,10.0,...,298.0,25.9,NaN,1004.6,NaN,2022-03-12,7.000000,11.042857,308.0,0
991237,washingtondc,38.88535,-77.038628,0.0,2022,2022-03-21,80,-1.4,-6.0,6.7,...,NaN,17.6,NaN,NaN,NaN,2022-03-13,4.333333,8.457143,306.6,0
991238,washingtondc,38.88535,-77.038628,0.0,2022,2022-03-21,80,6.7,0.0,16.1,...,190.0,13.0,NaN,1029.1,NaN,2022-03-14,3.466667,6.400000,313.3,0
991239,washingtondc,38.88535,-77.038628,0.0,2022,2022-03-21,80,11.8,5.6,21.7,...,183.0,11.5,NaN,1024.0,NaN,2022-03-15,5.700000,6.428571,325.1,1
991240,washingtondc,38.88535,-77.038628,0.0,2022,2022-03-21,80,14.2,7.2,22.8,...,157.0,7.9,NaN,1021.3,NaN,2022-03-16,10.900000,7.471429,339.3,2
991241,washingtondc,38.88535,-77.038628,0.0,2022,2022-03-21,80,12.9,10.0,16.1,...,15.0,12.2,NaN,1014.8,NaN,2022-03-17,12.966667,8.371429,352.2,3
991242,washingtondc,38.88535,-77.038628,0.0,2022,2022-03-21,80,14.1,8.3,23.3,...,175.0,13.3,NaN,1015.5,NaN,2022-03-18,13.733333,9.057143,366.3,4
991243,washingtondc,38.88535,-77.038628,0.0,2022,2022-03-21,80,18.2,13.9,24.4,...,196.0,22.3,NaN,1008.4,NaN,2022-03-19,15.066667,10.928571,384.5,5
991244,washingtondc,38.88535,-77.038628,0.0,2022,2022-03-21,80,14.9,10.0,16.7,...,305.0,24.1,NaN,1013.3,NaN,2022-03-20,15.733333,13.257143,399.4,6
991245,washingtondc,38.88535,-77.038628,0.0,2022,2022-03-21,80,13.5,6.7,21.7,...,271.0,13.3,NaN,1021.4,NaN,2022-03-21,15.533333,14.228571,412.9,7


In [295]:
df_ext.to_csv('data_ext/all_locations_weather_daily.csv', index=False) 

### Using monthly data for regression

In [381]:
def get_weather_monthly(row, cols):
    # get dataframe with monthly weather info for the whole year
    start = datetime(row.year, 1, 1) 
    end = datetime(row.year, 12, 31)
    location = Point(row.lat, row.long, row.alt)

    #if there are no data for this year, skip it
    try:
        df_weather = Monthly(location, start, end)
        df_weather = df_weather.fetch()
        if df_weather.shape[0]==0:
            print("No data for ", row.location, ' at ', row.year)
            return pd.DataFrame()
    except Exception as e:
        print("No data at ", row.location, ' for the whole year ', row.year)
        return pd.DataFrame()
    df_weather=df_weather.loc[:,["tavg","tmin","tmax","prcp"]]
    df_weather["month"]=df_weather.index.strftime("%B")
    
    #sometimes available only average temperature; sometimes  only minimum and maximum temperature
    #replace NA's for average temperature, using minimum and maximum temperature
    df_weather.tavg.fillna((df_weather.tmin+ df_weather.tmax)/2, inplace=True)
    #replace NA's for minimum and maximum temperature using average temperature
    df_weather["tmin"].fillna(df_weather.tavg, inplace=True)
    df_weather["tmax"].fillna(df_weather.tavg, inplace=True)
    
    # merge weatgher data with location
    df_loc = pd.DataFrame(columns=cols)
    df_loc.loc[0] = row
    df = pd.concat([df_loc,
               pd.pivot_table(df_weather, values='tavg', columns='month').add_suffix('_tavg').reset_index(drop=True),
               pd.pivot_table(df_weather, values='tmin', columns='month').add_suffix('_tmin').reset_index(drop=True),
               pd.pivot_table(df_weather, values='tmax', columns='month').add_suffix('_tmax').reset_index(drop=True),
               pd.pivot_table(df_weather, values='prcp', columns='month').add_suffix('_prcp').reset_index(drop=True)],
               axis=1)
    return df

In [382]:
import calendar
months = list(calendar.month_name)[1:]
cols = list(df_bloom_unq.columns) +[f"{i}_tavg" for i in months]+[f"{i}_tmin" for i in months]+ \
       [f"{i}_tmax" for i in months]+[f"{i}_prcp" for i in months]
df_monthly = pd.DataFrame(columns=cols)
for index, row in df_bloom_unq.iterrows():
    df_cur = get_weather_monthly(row, df_bloom_unq.columns)
    df_monthly = df_monthly.append(df_cur, ignore_index=True)
    #if index>70:
    #    break

No data for  Japan/Rumoi  at  2005
No data for  Japan/Iwamizawa  at  1990
No data for  Japan/Iwamizawa  at  2005
No data for  Japan/Obihiro  at  2005
No data for  Japan/Muroran  at  2005
No data for  Japan/Muroran  at  2005
No data for  Japan/Esashi  at  2005
No data for  Japan/Kutchan  at  1990
No data for  Japan/Kutchan  at  2005
No data for  Japan/Mombetsu  at  1990
No data for  Japan/Mombetsu  at  2005
No data for  Japan/Hiroo  at  1990
No data for  Japan/Hiroo  at  1991
No data for  Japan/Shinjo  at  1990
No data for  Japan/Shinjo  at  1991
No data for  Japan/Morioka  at  2005
No data for  Japan/Sakata  at  1953
No data for  Japan/Sakata  at  1954
No data for  Japan/Sakata  at  1955
No data for  Japan/Sakata  at  1956
No data for  Japan/Sakata  at  1957
No data for  Japan/Sakata  at  1958
No data for  Japan/Sakata  at  1959
No data for  Japan/Sakata  at  1960
No data for  Japan/Sakata  at  1961
No data for  Japan/Sakata  at  1962
No data for  Japan/Sakata  at  1963
No data for  Ja

No data for  Japan/Fukui  at  1990
No data for  Japan/Fukui  at  2005
No data for  Japan/Takayama  at  1990
No data for  Japan/Takayama  at  2005
No data for  Japan/Kumagaya  at  1990
No data for  Japan/Tsuruga  at  1990
No data for  Japan/Tsuruga  at  2005
No data for  Japan/Iida  at  2005
No data for  Japan/Kofu  at  2005
No data for  Japan/Hamamatsu  at  1990
No data for  Japan/Shizuoka  at  1990


No data for  Japan/Miyakejima  at  1992
No data for  Japan/Miyakejima  at  1993


No data for  Japan/Toyooka  at  1990
No data for  Japan/Toyooka  at  1991
No data for  Japan/Toyooka  at  1992
No data for  Japan/Toyooka  at  1993
No data for  Japan/Toyooka  at  1994
No data for  Japan/Toyooka  at  1995
No data for  Japan/Hikone  at  1990
No data for  Japan/Hikone  at  1991
No data for  Japan/Hikone  at  1992
No data for  Japan/Hikone  at  2005


No data for  Japan/Okayama  at  1990
No data for  Japan/Okayama  at  1991
No data for  Japan/Okayama  at  1992
No data for  Japan/Okayama  at  2005
No data for  Japan/Sumoto  at  1990
No data for  Japan/Wakayama  at  1990
No data for  Japan/Saga  at  1990
No data for  Japan/Saga  at  1991
No data for  Japan/Saga  at  2005
No data for  Japan/Nobeoka  at  1990
No data for  Japan/Nobeoka  at  1991
No data for  Japan/Nobeoka  at  1992
No data for  Japan/Nobeoka  at  1993
No data for  Japan/Fukue  at  1953
No data for  Japan/Fukue  at  1954
No data for  Japan/Fukue  at  1955
No data for  Japan/Fukue  at  1956
No data for  Japan/Fukue  at  1957
No data for  Japan/Fukue  at  1958
No data for  Japan/Fukue  at  1959
No data for  Japan/Fukue  at  1960
No data for  Japan/Fukue  at  1961
No data for  Japan/Uwajima  at  1990
No data for  Japan/Uwajima  at  1991
No data for  Japan/Uwajima  at  1992
No data for  Japan/Uwajima  at  2005
No data for  Japan/Kochi  at  2005
No data for  Japan/Kochi  at  

...

No data for  South Korea/Gunsan  at  1993
No data for  South Korea/Gunsan  at  1994
No data for  South Korea/Gunsan  at  2015


No data for  South Korea/Daegu  at  1980
No data for  South Korea/Daegu  at  1981
No data for  South Korea/Daegu  at  1982
No data for  South Korea/Daegu  at  1983
No data for  South Korea/Daegu  at  1984
No data for  South Korea/Daegu  at  1985
No data for  South Korea/Daegu  at  1986
No data for  South Korea/Daegu  at  1987
No data for  South Korea/Daegu  at  1988
No data for  South Korea/Daegu  at  1989
No data for  South Korea/Daegu  at  1990
No data for  South Korea/Daegu  at  1991
No data for  South Korea/Daegu  at  1992
No data for  South Korea/Daegu  at  1993
No data for  South Korea/Daegu  at  1994
No data for  South Korea/Daegu  at  1995
No data for  South Korea/Daegu  at  2015
No data for  South Korea/Jeonju  at  1980
No data for  South Korea/Jeonju  at  1981
No data for  South Korea/Jeonju  at  1982
No data for  South Korea/Jeonju  at  1984
No data for  South Korea/Jeonju  at  1985
No data for  South Korea/Jeonju  at  1986
No data for  South Korea/Jeonju  at  1987
No data f

No data for  South Korea/Wando  at  1980
No data for  South Korea/Wando  at  1981
No data for  South Korea/Wando  at  1982
No data for  South Korea/Wando  at  1983
No data for  South Korea/Wando  at  1984
No data for  South Korea/Wando  at  1985
No data for  South Korea/Wando  at  1986
No data for  South Korea/Wando  at  1987
No data for  South Korea/Wando  at  1988
No data for  South Korea/Wando  at  1989
No data for  South Korea/Wando  at  1990
No data for  South Korea/Wando  at  1991
No data for  South Korea/Wando  at  1992
No data for  South Korea/Wando  at  1993
No data for  South Korea/Wando  at  1994
No data for  South Korea/Wando  at  1995
No data for  South Korea/Wando  at  1996
No data for  South Korea/Wando  at  2015
No data for  South Korea/Seongsan  at  1980
No data for  South Korea/Seongsan  at  1981
No data for  South Korea/Seongsan  at  1982
No data for  South Korea/Seongsan  at  1983
No data for  South Korea/Seongsan  at  1984
No data for  South Korea/Seongsan  at  198

No data for  South Korea/Jangheung  at  1981
No data for  South Korea/Jangheung  at  1982
No data for  South Korea/Jangheung  at  1983
No data for  South Korea/Jangheung  at  1984
No data for  South Korea/Jangheung  at  1985
No data for  South Korea/Jangheung  at  1986
No data for  South Korea/Jangheung  at  1987
No data for  South Korea/Jangheung  at  1988
No data for  South Korea/Jangheung  at  1989
No data for  South Korea/Jangheung  at  1990
No data for  South Korea/Jangheung  at  1991
No data for  South Korea/Jangheung  at  1992
No data for  South Korea/Jangheung  at  1993
No data for  South Korea/Jangheung  at  1994
No data for  South Korea/Goheung  at  1980
No data for  South Korea/Goheung  at  1981
No data for  South Korea/Goheung  at  1982
No data for  South Korea/Goheung  at  1983
No data for  South Korea/Goheung  at  1984
No data for  South Korea/Goheung  at  1985
No data for  South Korea/Goheung  at  1986
No data for  South Korea/Goheung  at  1987
No data for  South Korea/G

No data for  South Korea/Geochang  at  1980
No data for  South Korea/Geochang  at  1981
No data for  South Korea/Geochang  at  1982
No data for  South Korea/Geochang  at  1983
No data for  South Korea/Geochang  at  1984
No data for  South Korea/Geochang  at  1985
No data for  South Korea/Geochang  at  1986
No data for  South Korea/Geochang  at  1987
No data for  South Korea/Geochang  at  1988
No data for  South Korea/Geochang  at  1989
No data for  South Korea/Geochang  at  1990
No data for  South Korea/Geochang  at  1991
No data for  South Korea/Geochang  at  1992
No data for  South Korea/Geochang  at  1993
No data for  South Korea/Geochang  at  1994
No data for  South Korea/Hapcheon  at  1980
No data for  South Korea/Hapcheon  at  1981
No data for  South Korea/Hapcheon  at  1982
No data for  South Korea/Hapcheon  at  1984
No data for  South Korea/Hapcheon  at  1985
No data for  South Korea/Hapcheon  at  1986
No data for  South Korea/Hapcheon  at  1987
No data for  South Korea/Hapcheo

In [384]:
df_monthly.tail(10)

,location,lat,long,alt,year,bloom_date,bloom_doy,January_tavg,February_tavg,March_tavg,...,March_prcp,April_prcp,May_prcp,June_prcp,July_prcp,August_prcp,September_prcp,October_prcp,November_prcp,December_prcp
10196,washingtondc,38.88535,-77.038628,0.0,2013,2013-04-09,99,4.6,3.5,6.6,...,71.0,70.0,72.0,253.0,112.0,34.0,31.0,159.0,74.0,141.0
10197,washingtondc,38.88535,-77.038628,0.0,2014,2014-04-10,100,0.2,3.3,6.1,...,108.0,164.0,126.0,84.0,119.0,86.0,28.0,89.0,67.0,89.0
10198,washingtondc,38.88535,-77.038628,0.0,2015,2015-04-10,100,1.9,-1.1,7.0,...,102.6,86.8,48.9,303.4,127.4,29.4,54.6,77.3,53.4,123.0
10199,washingtondc,38.88535,-77.038628,0.0,2016,2016-03-25,85,1.6,4.3,11.6,...,29.5,52.2,143.6,93.5,79.7,70.9,63.5,22.9,19.3,66.5
10200,washingtondc,38.88535,-77.038628,0.0,2017,2017-03-25,84,5.4,8.4,8.1,...,81.2,66.6,141.1,28.7,232.4,116.4,36.4,51.2,51.0,12.9
10201,washingtondc,38.88535,-77.038628,0.0,2018,2018-04-05,95,1.6,7.1,6.1,...,48.8,91.3,221.7,132.3,247.2,131.9,247.2,77.8,192.4,147.8
10202,washingtondc,38.88535,-77.038628,0.0,2019,2019-04-01,91,2.9,5.7,8.2,...,102.0,57.0,126.0,109.0,165.0,51.0,7.0,169.0,35.0,83.0
10203,washingtondc,38.88535,-77.038628,0.0,2020,2020-03-20,80,5.8,6.6,11.8,...,59.0,160.0,63.0,89.0,165.0,222.0,141.0,124.0,156.0,126.0
10204,washingtondc,38.88535,-77.038628,0.0,2021,2021-03-28,87,3.7,3.2,10.7,...,97.0,56.0,96.0,140.0,108.0,230.0,103.0,83.0,25.0,16.1
10205,washingtondc,38.88535,-77.038628,0.0,2022,2022-03-21,80,1.3,5.5,10.1,...,70.3,96.9,161.7,75.3,193.5,61.6,57.6,70.7,67.3,98.2


In [385]:
df_monthly.to_csv('data_ext/all_locations_weather_monthly.csv', index=False) 